In [ ]:
!pip install py-midi
!pip install pretty_midi
!pip install music21
!pip install gensim

In [ ]:
pip install --upgrade tensorflow

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import midi_statistics
import utils
from matplotlib.patches import Polygon
import os
from gensim.models import Word2Vec
from sklearn.neighbors import KernelDensity

In [4]:
tf.__version__

'2.1.0'

In [5]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

W0319 11:01:39.587549 139884901349184 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/compat/v2_compat.py:88: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term


In [6]:
syll_model_path = './enc_models/syllEncoding_20190419.bin'
word_model_path = './enc_models/wordLevelEncoder_20190419.bin'

syllModel = Word2Vec.load(syll_model_path)
wordModel = Word2Vec.load(word_model_path)

# Lyrics preprocessing

In [ ]:
!pip install pyhyphen
!pip install nltk

In [8]:
import nltk

nltk.download('punkt')

from nltk import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [9]:
import hyphen
 
lang = 'en_US'
if not hyphen.dictools.is_installed(lang): 
  hyphen.dictools.install(lang)

hyph = hyphen.Hyphenator(lang)
print([hyph.syllables(word) for word in word_tokenize('people')])

[['peo', 'ple']]


In [10]:
lyrics = [['Must','Must'],['have','have'],['been','been'],['love','love'],
          ['but','but'],['its','its'],['o','over'],['ver','over'],['now','now'],['lay','lay'],['a','a'],
          ['whis','whisper'],['per','whisper'],['on','on'],['my','my'],['pil','pillow'],['low','pillow']]
# lyrics = [['Then','Then'],['the','the'],['rain','rainstorm'],['storm','rainstorm'],['came','came'],
#           ['ov','over'],['er','over'],['me','me'],['and','and'],['i','i'],['felt','felt'],['my','my'],
#           ['spi','spirit'],['rit','spirit'],['break','break']]
# lyrics = [['E','Everywhere'],['very','Everywhere'],['where','Everywhere'],['I','I'],['look','look'],
#          ['I','I'],['found','found'],['you','you'],['look','looking'],['king','looking'],['back','back']]
# lyrics = [['You','You'],['turn','turn'],['my','my'],['nights','nights'],
#           ['in','into'],['in','into'],['days','days'],['Lead','Lead'],['me','me'],['mys','mysterious'],['te','mysterious'],
#           ['ri','mysterious'],['ous','mysterious'],['ways','ways']]

In [11]:
lyrics_text = """
It pains me to say... you made my life so much more. . What an impossible dream. I can't imagine what will happen. . So many people that I know. All I could ever imagine. To be like you. . What am I all wrong??. Oh... what am I all wrong?. . What am I all wrong?. Oh... what am I all wrong?. . So many people that I know. All I can ever imagine. To be like you. . What am I all wrong?. Oh... what am I all wrong?."All I can ever imagine (All I can imagine). To be like you. . What am I all wrong?. Oh... what am I all wrong?. . So many people that I know. All I can ever imagine. To be like you. . What am I all wrong?. Ah... what am I all wrong?. . So many people that I know. All I can ever imagine. To be like you

I could stay or go. I could go any time I wanted. I could always get away. I could get out of my head. I could never ever stop thinking. . I could feel the time is right. I could feel the heart is still blue. The love that I think I need. It makes me sad. . I could feel the time is right. I could feel the heart is still blue. The love that I think I need. It makes me sad. . A million stars, a million days. I can only pray that someday. I can go any time I want. I can always get away. I could never ever stop thinking. . I could feel the time is right. I could feel the heart is still blue. The love that I think I need. It makes me sad. . I could feel the time is right. I could feel the heart is still blue. The love that I think I need. It makes me sad. . Oh my heart beat and shakes. It's like a million angels. That I pray to keep. . Oh my heart is beating with love and hope. It's like a million angels. That I pray to keep. . Oh my heart beats and shakes. It's like a million angels. That I pray to keep. . Oh my heart beats and shakes. It's like a million angels. That I pray to keep. . It's that very same feeling. That the stars really are blue. And I pray to keep. . I could feel the time is right. I could feel the heart is still blue. The love that I think I need. It makes me sad. . I could feel the time is right. I could feel the heart is still blue. The love that I think I need. It makes me sad. . I could feel the time is right. I could feel the heart is still blue. The love that I think I need. It makes me sad

You're taking me to another dimension. Every time you touch me. I feel what I need. The gravity is real. . I knew we were nothing. All that we could be. But I knew all that we could be. . But I knew we couldn't make anything else go away. It feels so good to see you. . I can't help but smile, I gotta see how that is. And I know that
"""

In [ ]:
lyrics = []
for word in word_tokenize(lyrics_text):
  if word in ("n't", "'s"):  # e.g. can't doesn't it's
    lyrics[-1][0] += word
    lyrics[-1][1] += word
  elif word.isalpha():
    syls = hyph.syllables(word)
    if len(syls) == 0:
      syls = [word]
    for syl in syls:
        lyrics.append([syl, word])

print(lyrics)

# Inference

In [14]:
moving_window_step = 10
input_tensor_size = 20

length_song = len(lyrics)

all_flat_conditions = []

idx = 0
lyrics_covered = False
while not lyrics_covered and idx * moving_window_step < length_song:
    offset = idx * moving_window_step
    
    cond = []
    for i in range(offset, offset + input_tensor_size):
        if i < length_song:
            syll2Vec = syllModel.wv[lyrics[i][0]]
            word2Vec = wordModel.wv[lyrics[i][1]]
            cond.append(np.concatenate((syll2Vec,word2Vec)))
        else:
            cond.append(np.concatenate((syll2Vec,word2Vec)))

    all_flat_conditions.append([element for sublist in cond for element in sublist])

    if offset + input_tensor_size >= length_song:
        lyrics_covered = True
    else:
        idx += 1

In [19]:
model_path = 'saved_gan_models/1August/epochs_models/model_epoch395'

output = []

with tf.Session(graph=tf.Graph()) as sess:
    tf.saved_model.loader.load(sess, [], model_path)
    graph = tf.get_default_graph()
    keep_prob = graph.get_tensor_by_name("model/keep_prob:0")
    input_metadata = graph.get_tensor_by_name("model/input_metadata:0")
    input_songdata = graph.get_tensor_by_name("model/input_data:0")
    output_midi = graph.get_tensor_by_name("output_midi:0")
    
    feed_dict = {}
    feed_dict[keep_prob.name] = 1.0
    
    for idx, flattened_cond in enumerate(all_flat_conditions):
        condition = []
        feed_dict[input_metadata.name] = condition
        feed_dict[input_songdata.name] = np.random.uniform(size=(1, input_tensor_size, 3))
        condition.append(np.split(np.asarray(flattened_cond), input_tensor_size))
        feed_dict[input_metadata.name] = condition

        generated_features = sess.run(output_midi, feed_dict)

        sample = [x[0, :] for x in generated_features]
        sample = midi_statistics.tune_song(utils.discretize(sample))
        
        output += sample[-moving_window_step:]
        # output += sample[:moving_window_step]

midi_pattern = utils.create_midi_pattern_from_discretized_data(output)
midi_pattern.write(f"output.mid")

In [25]:
repr(type("abc"))

"<class 'str'>"

In [21]:
import music21

# fctr = 1.25 # scale (in this case stretch) the overall tempo by this factor
fctr = (3 * 60 + 20) / (8 * 60 + 53)   # reduce from 8:53 to 3:20

score = music21.converter.parse('output.mid')
newscore = score.scaleOffsets(fctr).scaleDurations(fctr)

newscore.write('midi','output_fast.mid') 

'output_fast.mid'

# MIDI TO WAV

In [86]:
! pip install midi2audio
! apt-get update
! apt-get install fluidsynth

You should consider upgrading via the 'pip install --upgrade pip' command.


In [98]:
from midi2audio import FluidSynth
fs = FluidSynth(sample_rate=44100)
fs.midi_to_audio('output.mid', 'output.wav')

# import midi2audio as m2a

# INSTRUMENT MANIPULATIONS

In [93]:
# https://stackoverflow.com/questions/14448380/how-do-i-read-a-midi-file-change-its-instrument-and-write-it-back

from music21 import converter, instrument

for instr in [instrument.Violin(), instrument.Guitar(), instrument.Accordion()]:
    print(instr)

    s = converter.parse('output.mid')

    for el in s.recurse():
        if 'Instrument' in el.classes: # or 'Piano'
            el.activeSite.replace(el, instr)

    ### or if there are no patch changes currently defined:
    # for p in s.parts:
    #     p.insert(0, instr)

    s.write('midi', f'output_{instr}.mid')

Violin
Guitar
Accordion


# PLAY IT

In [68]:
!pip install pygame

     |████████████████████████████████| 11.4MB 1.7MB/s 
You should consider upgrading via the 'pip install --upgrade pip' command.


In [70]:
from music21 import midi

# mf = midi.MidiFile()
# mf.open("test.mid")
# mf.read()
# mf.close()
# s = midi.translate.midiFileToStream(mf)
# s.show('midi')

def play_midi(file_path):
    mf = midi.MidiFile()
    mf.open(file_path)
    mf.read()
    mf.close()
    s = midi.translate.midiFileToStream(mf)
    sp = midi.realtime.StreamPlayer(s)
    sp.play()

# play_midi("test.mid")

# Text-to-Speech

In [2]:
!pip install gtts

     |████████████████████████████████| 112kB 2.8MB/s 
  Created wheel for gtts-token: filename=gTTS_token-1.1.3-cp36-none-any.whl size=4478 sha256=77e87fbba71138839c93cc8af3b881d8fe7091639b0114087d00f10015139d21
  Stored in directory: /root/.cache/pip/wheels/dd/11/61/33f7e51bf545e910552b2255eead2a7cd8ef54064b46dceb34
Successfully built gtts-token
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
from gtts import gTTS 
import os

In [5]:
language = "en"

In [6]:
# lyrics = "Global warming is the long-term rise in the average temperature of the Earth’s climate system"

In [ ]:
speech = gTTS(text = lyrics_text, lang = language, slow = False)

In [100]:
speech.save("tts.mp3")

In [101]:
import IPython.display as ipd
ipd.Audio("tts.mp3")

In [ ]:
!pip install --upgrade google-cloud-texttospeech

In [56]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "Speech-to-text-578390455ec1.json"

from google.cloud import texttospeech
import re

# Instantiates a client
client = texttospeech.TextToSpeechClient()

# Set the text input to be synthesized
# synthesis_input = texttospeech.types.SynthesisInput(text="Do no evil!")
synthesis_input = texttospeech.types.SynthesisInput(text=re.sub(r"[^\w\d'\s]+", '', lyrics_text))  # remove all punctuation but apostrophes and spaces

# Build the voice request, select the language code ("en-US") 
# ****** the NAME
# and the ssml voice gender ("neutral")
voice = texttospeech.types.VoiceSelectionParams(
    language_code='en-US',
    # name='en-US-Wavenet-C',
    ssml_gender=texttospeech.enums.SsmlVoiceGender.MALE)

# Select the type of audio file you want returned
audio_config = texttospeech.types.AudioConfig(
    audio_encoding=texttospeech.enums.AudioEncoding.MP3)

# Perform the text-to-speech request on the text input with the selected
# voice parameters and audio file type
response = client.synthesize_speech(synthesis_input, voice, audio_config)

# The response's audio_content is binary.
with open('google_tts_output.mp3', 'wb') as out:
    # Write the response to the output file.
    out.write(response.audio_content)

In [57]:
import IPython.display as ipd
ipd.Audio("google_tts_output.mp3")

In [53]:
import IPython.display as ipd
ipd.Audio("google_tts_output_male.mp3")

In [58]:
!apt-get update
!apt-get install musescore

!git clone https://github.com/mathigatti/midi2voice

Cloning into 'midi2voice'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 265 (delta 5), reused 11 (delta 3), pack-reused 251
Receiving objects: 100% (265/265), 155.19 MiB | 31.26 MiB/s, done.
Resolving deltas: 100% (75/75), done.


In [67]:
with open('midi2voice/lyrics.txt', 'wt') as f:
    f.write(lyrics_text)

In [73]:
import os
os.environ["QT_QPA_PLATFORM"] = "offscreen"  # required for musescore command to run without X DISPLAY

import sys
sys.path.append('midi2voice')

textFilePath = 'midi2voice/lyrics.txt'
midiPath = 'output_Accordion.mid'
sex = "female"
tempo = 80

sys.argv = ['midi2voice/midi2voice.py', textFilePath, midiPath, sex, tempo]

import midi2voice.midi2voice as m2v

Running voice renderization
Text:
['\nit', 'pains', 'me', 'to', 'say', 'you', 'made', 'my', 'life', 'so', 'much', 'more', 'what', 'an', 'impossible', 'dream', 'i', 'cant', 'imagine', 'what', 'will', 'happen', 'so', 'many', 'people', 'that', 'i', 'know', 'all', 'i', 'could', 'ever', 'imagine', 'to', 'be', 'like', 'you', 'what', 'am', 'i', 'all', 'wrong', 'oh', 'what', 'am', 'i', 'all', 'wrong', 'what', 'am', 'i', 'all', 'wrong', 'oh', 'what', 'am', 'i', 'all', 'wrong', 'so', 'many', 'people', 'that', 'i', 'know', 'all', 'i', 'can', 'ever', 'imagine', 'to', 'be', 'like', 'you', 'what', 'am', 'i', 'all', 'wrong', 'oh', 'what', 'am', 'i', 'all', 'wrongall', 'i', 'can', 'ever', 'imagine', '(all', 'i', 'can', 'imagine)', 'to', 'be', 'like', 'you', 'what', 'am', 'i', 'all', 'wrong', 'oh', 'what', 'am', 'i', 'all', 'wrong', 'so', 'many', 'people', 'that', 'i', 'know', 'all', 'i', 'can', 'ever', 'imagine', 'to', 'be', 'like', 'you', 'what', 'am', 'i', 'all', 'wrong', 'ah', 'what', 'am', 'i', 'a

Exception: No wav file found on sinsy.jp

In [ ]:
main(textFilePath, midiPath, sex, tempo)